In [3]:
# Import the os module to interact with the operating system environment variables
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Read PDF File

In [4]:
file_path = "heartstart RAG.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

300


In [5]:
print(docs[0].page_content)
print(docs[0].metadata)

Instructions for Use
HeartStart Intrepid
Monitor /D efibrillator
867172 
 English

{'source': 'heartstart RAG.pdf', 'page': 0}


# Chunking Text

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()


C:\Users\Hrisi\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Create Pinecone index

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import time
INDEX_NAME = "chatbotqa-index"
USE_SERVERLESS = True
spec = ServerlessSpec(cloud = 'aws', region = 'us-east-1')
# Initialize a ServerlessSpec object for AWS with the specified region
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
# Check if the index already exists in the current PC (presumably a database or similar)
if USE_SERVERLESS:
    if INDEX_NAME in pc.list_indexes().names():
        # If the index exists, print a message indicating its existence
        print(f"Index `{INDEX_NAME}` already exists")
        INDEX = pc.Index(INDEX_NAME)        
        # Print detailed statistics about the existing index
        print(INDEX.describe_index_stats())
        
    # If the index does not exist, proceed to create a new one
    else:
        # Create a new index with specific parameters        
        pc.create_index(
            name=INDEX_NAME,
            dimension=768,
            metric="cosine",
            spec=spec
        )
        
    # Wait for the index to be initialized before proceeding
    while not pc.describe_index(INDEX_NAME).status['ready']:
        # Sleep for 1 second to avoid overloading the system with requests
        time.sleep(1)
    
    # Once the index is ready, print a confirmation message
    print(f"Index with name `{INDEX_NAME}` is created")
    
    # Retrieve the newly created index object
    index = pc.Index(INDEX_NAME)
    
    # Print detailed statistics about the newly created index
    print(index.describe_index_stats())

Index with name `chatbotqa-index` is created
{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


# Insert into Pinecone Vector DB Index

In [57]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
        splits,
        index_name=INDEX_NAME,
        embedding=embeddings
    )

In [55]:
print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-mpnet-base-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


# Retreive context

In [42]:
from langchain_pinecone import PineconeVectorStore
INDEX_NAME = "chatbotqa-index"
docsearch = PineconeVectorStore(index_name=INDEX_NAME, embedding=embeddings)

In [74]:
retriver = docsearch.as_retriever(search_kwargs={"k": 3})
results = retriver.invoke("How to install the battery?")


In [80]:
for doc in results:
    print(doc.metadata)

{'page': 32.0, 'source': 'heartstart RAG.pdf'}
{'page': 32.0, 'source': 'heartstart RAG.pdf'}
{'page': 4.0, 'source': 'heartstart RAG.pdf'}


# Model Setup

In [44]:
from langchain.llms import HuggingFaceHub
import os
# Define the repo ID and connect to Mixtral model on Huggingface
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceHub(
  repo_id=repo_id, 
  model_kwargs={"temperature": 0.8, "top_k": 50}, 
  huggingfacehub_api_token=os.getenv('HUGGING_FACE_API_TOKEN')
)

# Create Prompt 

In [117]:
from langchain import PromptTemplate

template = """ <s> [INST] You are an expert in operating and maintaining the Philips HeartStart Intrepid monitor/defibrillator
Users will ask you questions about the device and how to maintain and operate it. 
Use following piece of context to answer the question and return only your response.
Use only the context to answer, do not give references. Simply answer the question without editorial comments.
If you don't know the answer, just say you don't know. 
Keep the answer within 2 sentences and concise.
</s> [/INST]

Context: {context}
Question: {question}
Answer:
"""

prompt = PromptTemplate(
  template=template, 
  input_variables=["context", "question"]
)

# Chain everything together

In [95]:
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
rag_chain = RetrievalQA.from_chain_type(
            llm, retriever=docsearch.as_retriever(), chain_type_kwargs={"prompt": prompt}
        )
rag_chain.invoke("How to install the battery?")

C:\Users\Hrisi\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


{'query': 'How to install the battery?',
 'result': "\nYou are an expert in operating and maintaining the Philips HeartStart Intrepid monitor/defibrillator\nUsers will ask you questions about the device and how to maintain and operate it. \nUse following piece of context to answer the question and return only your response. \nIf you don't know the answer, just say you don't know. \nKeep the answer within 2 sentences and concise.\n\n\nContext: Battery and AC Power 2: Device Basics\n 19\n WARNING: Do not connect a LAN cable to the HeartStart Intrepid. The LAN port is for factory use only. \nBattery and AC Power\nThis section describes basics of power supply. See “Power” on page 31 for a detailed discussion. \nInstalling the Battery\n\uf0a5 To install the lithium ion battery:\n1 Align the battery in the battery compartment. Confirm the arrow on the Battery Tab is \npositioned at the bottom, see Figure 18. \n2 Push the battery into the battery compartment until the battery latch is locked 